In [1]:
!pip install langchain langchain-community langchain-core langchain-groq sentence-transformers faiss-cpu unstructured pymupdf gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.6/441.6 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.1 MB/s eta 0:00:00
   ━━━━━

In [ ]:
# ✅ INSTALL IF NEEDED:
# pip install langchain langchain-community langchain-core langchain-groq sentence-transformers faiss-cpu pymupdf gradio

import os
import gradio as gr
import traceback
import numpy as np
from pathlib import Path

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq

# ----------------------------------------
# ✅ SET YOUR GROQ API KEY
# ----------------------------------------
os.environ["GROQ_API_KEY"] = "your-api-key"  # ⬅️ Replace with your actual key!

# ----------------------------------------
# ✅ Utility Functions
# ----------------------------------------

def load_pdf(file_path):
    try:
        loader = PyMuPDFLoader(file_path)
        return loader.load()
    except Exception as e:
        raise ValueError(f"❌ Failed to load PDF: {str(e)}")

def split_docs(docs):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1200,
        chunk_overlap=200
    )
    return splitter.split_documents(docs)

def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs if doc.page_content.strip() != ""])

def create_vectorstore(docs):
    embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
    vector_store = FAISS.from_documents(docs, embedding=embeddings)
    return vector_store, embeddings

def is_question_relevant(query, vector_store, embeddings, threshold=0.5):
    query_embedding = embeddings.embed_query(query)
    query_vector = np.array([query_embedding], dtype='float32')
    D, _ = vector_store.index.search(query_vector, k=1)
    return D[0][0] < (1 - threshold)

def build_rag_chain(retriever):
    prompt = """
You are a helpful financial assistant answering questions based only on the provided 10-Q document context.
If the context does not contain an answer, say: "❌ No relevant information found in the document."

Answer in bullet points if appropriate.

Question: {question}
Context: {context}
Answer:
    """
    model = ChatGroq(model_name="llama3-8b-8192", api_key=os.environ["GROQ_API_KEY"])
    template = ChatPromptTemplate.from_template(prompt)

    return (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | template
        | model
        | StrOutputParser()
    )

# ----------------------------------------
# ✅ Gradio Logic
# ----------------------------------------

vector_store = None
embeddings = None
rag_chain = None
retriever = None

def upload_and_process(file):
    global vector_store, embeddings, rag_chain, retriever

    try:
        file_path = file.name
        raw_docs = load_pdf(file_path)
        if not raw_docs:
            return "❌ No content could be extracted from the PDF."

        docs = split_docs(raw_docs)
        if not docs:
            return "❌ Could not split document into chunks."

        vector_store, embeddings = create_vectorstore(docs)
        retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={'k': 5})
        rag_chain = build_rag_chain(retriever)

        return "✅ Document processed successfully! You can now ask questions."

    except Exception as e:
        return f"❌ Upload Error:\n{str(e)}\n\n{traceback.format_exc()}"

def ask_question(question):
    global vector_store, embeddings, rag_chain, retriever

    if rag_chain is None:
        return "❌ Please upload and process a document first."

    try:
        if not is_question_relevant(question, vector_store, embeddings, threshold=0.5):
            return "❌ No relevant information found in the document."

        docs = retriever.invoke(question)
        context = format_docs(docs)

        if not context or len(context.strip()) < 50:
            return "❌ Couldn’t retrieve sufficient context from the document."

        # ✅ Debug log
        print("\n--- Retrieved Context ---\n", context, "\n--------------------------\n")

        response = ""
        for chunk in rag_chain.stream(question):
            response += chunk

        return response or "❌ Model returned empty response."

    except Exception as e:
        return f"❌ Answering Error:\n{str(e)}\n\n{traceback.format_exc()}"

# ----------------------------------------
# ✅ Gradio Interface
# ----------------------------------------

with gr.Blocks() as demo:
    gr.Markdown("## 📊 Financial RAG Assistant ")

    with gr.Row():
        with gr.Column():
            file_input = gr.File(label="📁 Upload 10-Q PDF", file_types=[".pdf"])
            upload_btn = gr.Button("📄 Process Document")
            upload_status = gr.Textbox(label="Upload Status", lines=2)

        with gr.Column():
            question_input = gr.Textbox(label="❓ Ask a Question", placeholder="e.g. What was Amazon’s net income in Q3 2024?")
            ask_btn = gr.Button("🔍 Get Answer")
            answer_output = gr.Textbox(label="📢 Answer", lines=10)

    upload_btn.click(fn=upload_and_process, inputs=[file_input], outputs=[upload_status])
    ask_btn.click(fn=ask_question, inputs=[question_input], outputs=[answer_output])

demo.launch()


* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://ac80703f6fce076012.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
